# Coastal Systems Notebook 2 (week 3): The Tide

In chapter 3 of the book and slides, we learned that the tide is generated through the interplay between gravitational attraction forces in the Earth-Moon and Earth-Sun systems respectively. In this notebook we will explore these concepts in a more interactive way, with some exercises, visualizations, and code. We will not repeat the whole theory, however, so make sure you have followed the lectures for this week and read the relevant pages in chapter 3 of the book.

We will explore the different aspects of the tide through this Jupyter notebook and use python to create figures, make computations, and analyse data. So, please make sure you activate the coastal environment before running the notebook. Then run the cell below, where we import the packages used in this notebook.

In [ ]:
import pathlib
import sys

# make coastpy library importable by appending root to path
cwd = pathlib.Path().resolve()
proj_dir = cwd.parent.parent  # this is the root of the CoastalCodeBook
sys.path.append(str(proj_dir / "src"))
DATA_DIR = proj_dir / "data"

from warnings import filterwarnings
import ipywidgets as widgets
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## 1. The equilibrium tide

First we will have a look at the generation of the tide. To start, we are gonna look at the gravitational forces between the earth, moon, and sun. As you have learned, the tide is generated by the so-called *differential pull*, meaning the difference in gravitational pull on water masses at different locations on the Earth. To make calculations, we first need a function that can compute the gravitational pull between two celestial bodies (e.g. the Earth and Moon).

In [ ]:
### Functions
def grav_pull(m1, m2, d):
    """
    This function calculates the gravitational pull between two celestial bodies using Newton's law of universal gravitation.

    Parameters:
    m1 (float): mass of the first body in kg.
    m2 (float): mass of the second body in kg.
    d (float): distance between the two bodies in meters.

    Returns:
    g_pull (float): the gravitational pull between the two bodies in g

    """
    G = 6.67408e-11                 # universal gravitational constant
    g = 9.81                        # gravitational acceleration on the Earth
    g_pull = G * (m1 * m2) / d**2   # gravitational pull between two bodies
    
    # we divide the GP by g, so we get the gravitational pull in terms of g
    return g_pull / g 


The above function computes the gravitational pull between two celestial bodies. We can now use it to compute the forces between the earth and the sun/moon. Find the required values and fill them in below and then run the function, which will show you the forces. For this notebook, we will copmute the forces that are working on 1kg of Earth mass, to keep it simple. Hence, the mass we will fill in for the Earth will be 1kg. Also, we will express the forces in terms of the Earths own gravitational acceleration (g), which makes it easier to compare the magnitudes with each other.

In [ ]:
# Properties of Earth, Moon, and Sun
m_earth = 1         # mass of the earth in kg (we use 1 to compute the gravitational pull per kg of earth mass)
m_sun = 1.99e30     # mass of the sun in kg
m_moon = 7.35e22    # mass of the moon in kg
d_sun = 1.5e11      # distance between the centers of the sun and earth in meters
d_moon = 3.84e8     # distance between the centers of the moon and earth in meters
r = 6.37e6          # radius of the Earth in meters

# Now we compute the gravitational pull and print them
gp_sun = grav_pull(m_earth, m_sun, d_sun)
gp_moon = grav_pull(m_earth, m_moon, d_moon)

# Print out results
print("Gravitational pull from the Sun on the center of Earth:  %.2E g" % gp_sun)
print("Gravitational pull from the Moon on the center of Earth: %.2E g" % gp_moon)

As we can see, the gravitational pull of the sun is about two orders of magnitude larger than that of the moon, and both are much smaller than g. Yet, we know that these forces are somehow responsible for tidal ranges of multiple meters, and that the moon is actually responsible for about 70% of these tides. How come? As we can see in the formula, the gravitational pull is strongly dependent on the distance between the two celestial bodies. Moreover, since the Earth is a sphere, different locations on the Earth's surface will experience subtle differences in the gravitational pull, as the distance to the Sun/Moon varies slightly over the Earths surface. 

Let's use the above function to compute the gravitational pull from the sun at 8 different locations around the Earth's surface (A-H, see the figure below).

![earth_positions](./figures/02_earth_sun_gp.png)

In [ ]:
# First we compute the distance between the center of the sun and each position on Earth
distance = d_sun  # switch to d_moon to change the computation to moon
distances = {
    "X": distance,
    "A": np.sqrt(r**2 + distance**2),  # Same as point E
    "B": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (distance - r * np.cos(45 * np.pi / 180)) ** 2
    ),  # Same as point D
    "C": distance - r,
    "D": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (distance - r * np.cos(45 * np.pi / 180)) ** 2
    ),  # Same as point B
    "E": np.sqrt(r**2 + distance**2),  # Same as point A
    "F": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (r * np.cos(45 * np.pi / 180) + distance) ** 2
    ),  # Same as point H
    "G": distance + r,
    "H": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (r * np.cos(45 * np.pi / 180) + distance) ** 2
    ),  # Same as point F
}

# Now compute the gravitational pull for each point.
gp = {}
for pos in distances:
    gp[pos] = grav_pull(m_earth, m_sun, distances[pos])
    print("Gravitational pull at position %s: %.5E g" % (pos, gp[pos]))

# ang_A_E = np.arccos(distance/np.sqrt(r**2+distance**2))   # angle between point A and moon.
# dx,dy = gp_moon_A_E*np.cos(ang_A_E), gp_moon_A_E*np.sin(ang_A_E)

As we can see, the differences are very subtle (in the order of 10^-7 g). We also know that the gravitational pull is responsible for the Earth's rotation around the Sun. That would imply, however, that the rotation of the Earth varies over its body, along with the gravitational pull. Yet, we know that the Earth rotates around the Sun as a whole. Apparently, the Earth compensates for these subtle differences in gravitational pull so it can rotate around the Sun as one body. The water masses on the surface of the solid Earth, however, can move around and are not as fixed and rigid as the solid land masses. So, while the solid masses of the Earth remain virtually unchanged, its water masses are moved around by these subtle differences in gravitational pull.

So, let's visualize these differences. Earth's rotation is based on the gravitational pull that acts on its center, hence this will be our *reference pull*. For the other locations around the Earth, we will compute the difference to the reference pull, by subtracting the reference pull from the gravitational pull at each location. If we do this with the actual forces we computeed above we won't see the difference since they are very small. Therefore, we will visualize the forces out of proportion, i.e. we exaggerate the difference. Next to the difference in magnitude, there are also slight differences in the direction of the forces at different positions, as each force is directed towards the centre of the sun/moon. We will also visualize these out of proportion, so we can actually see the difference. Run the code cell below to get a visualization of the forces we talked about above.

In [ ]:
### VISUAIZATION FIGURE GRAV PULL
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Adding outer and inner circles
circ_out = mpatches.Circle((0, 0), 0.3, fill=True, color="grey", linewidth=2)
circ_in = mpatches.Circle((0, 0), 0.275, fill=True, color="lightgrey")
circ_c = mpatches.Circle((0, 0), 0.005, fill=True, color="black")
axs[0].add_patch(circ_out)
axs[0].add_patch(circ_in)
axs[1].add_patch(mpatches.Circle((0, 0), 0.3, fill=True, color="grey", linewidth=2))
axs[1].add_patch(mpatches.Circle((0, 0), 0.275, fill=True, color="lightgrey"))
axs[1].add_patch(circ_c)

# Coordinates and force components. The first two values are the coordinates of the position. 
# The 3rd and 4th value are the x/y components of the GP at each location (not to scale)
positions = {
    "X": [0,0,0.2,0],
    "A": [0, 0.3, 0.2, -0.025],
    "B": [0.3 * np.cos(45 * np.pi / 180), 0.3 * np.cos(45 * np.pi / 180), 0.25, -0.015],
    "C": [0.3, 0, 0.25, 0],
    "D": [0.3 * np.cos(45 * np.pi / 180), -0.3 * np.cos(45 * np.pi / 180), 0.25, 0.015],
    "E": [0, -0.3, 0.2, 0.025],
    "F": [-0.3 * np.cos(45 * np.pi / 180), -0.3 * np.cos(45 * np.pi / 180), 0.15, 0.01],
    "G": [-0.3, 0, 0.15, 0],
    "H": [-0.3 * np.cos(45 * np.pi / 180), 0.3 * np.cos(45 * np.pi / 180), 0.15, -0.01],
}

# Loop through each position
for pos in positions:
    # Center of the Earth
    if pos == "X":
        axs[0].add_patch(mpatches.Arrow(*positions[pos], width=0.025, color="k"))

    # Other positions
    else:
        # Gravitational pull (GP)
        axs[0].add_patch(
            mpatches.Arrow(
                *positions[pos][0:2],
                0.2,
                0,
                width=0.025,
                edgecolor="k",
                facecolor="None"
            )
        )  # GP at center of Earth
        axs[0].add_patch(
            mpatches.Arrow(*positions[pos], width=0.025, color="C3")
        )  # GP at each individual location

        # Differential pull
        diffpull = list(
            2 * (np.array(positions[pos][2:]) - np.array(positions["X"][2:]))
        )  # Subtract th GP at the center from the other locations
        axs[1].add_patch(
            mpatches.Arrow(*positions[pos][0:2], *diffpull, width=0.025, color="C2")
        )

# Adding center and text
titles = ["Gravitational pull", "Differential pull"]
for i in range(len(axs)):
    for pos in positions:
        coords = [x + 0.025 for x in positions[pos][0:2]]
        axs[i].text(*coords, pos, fontsize=12, ha="center", va="center")
    axs[i].axis("equal")
    axs[i].axis("off")
    axs[i].set_xlim(-0.6, 0.7)
    axs[i].set_ylim(-0.4, 0.4)
    axs[i].set_title(titles[i])

The red arrows indicate the gravitational pull at each individual location around Earth's surface. The black lined, transparent arrows represent the reference pull and are all equal to the black arrow at the center (X). If we subtract the black arrows from the red arrows at each location, we end up with the green arrows in the second plot, which we call the differential pull. Do the green arrows already give you an indication of the shape of the tides? 

Now, let's compute the actual differential pull at location C, for both the Earth-Sun and the Earth-Moon system. You can also check this calculation written out on page 124 of the book.

In [ ]:
dp_C_moon = grav_pull(m_earth, m_moon, d_moon - r) - grav_pull(m_earth, m_moon, d_moon)
dp_C_sun = grav_pull(m_earth, m_sun, distances["C"]) - grav_pull(m_earth, m_sun, d_sun)

print("Differential pull at C due to the moon: %E g" % dp_C_moon)
print("Differential pull at C due to the sun : %E g" % dp_C_sun)
print("Percentage of the total differential pull associated to the moon: %.1f%%"
    % (dp_C_moon / (dp_C_sun + dp_C_moon) * 100))
print("Percentage of the total differential pull associated to the sun : %.1f%%"
    % (dp_C_sun / (dp_C_sun + dp_C_moon) * 100))

Now we can see that the differential pull is actually larger for the moon than for the sun. Can you explain why this is the case (hint: look at the formula to compute the gravitational pull).

Still, both are several orders of magnitude smaller than the Earth's own attraction towards its centre, g. That means, that the component of the differential pull that is in line with Earth's center can be neglected, since its magnitude is negligible compared to g, and will therefore not make a noticable difference. The only thing we are left with then, are the *tangential components* of the differential pull. These are the forces that generate the tide by moving the water around the surface of the Earth. This results in the two bulges of water at either side of the Earth, in line with the Sun/Moon. Hence, if the Sun and Moon are aligned (full moon and new moon), the forces enhance each other and we get a spring tide (larger tidal range). On the other hand, if they are perpendicular to each other (1st and 3rd quarter moon), the lunar and solar differential pull work against each other and we get a neap tide. In the figure below you can play around with different constellations of the moon and sun, to see what the effect is on the total tidal shape.

The code below visualizes this concept. In the plot, the sun is fixed to the right side of the Earth (see also Figure 3.21 in the book). You can adjust the position of the Moon by setting its angle relative to the position of the sun. So if you set it at 90, the moon will be in a position 90 degrees counterclock-wise from the sun (i.e. they will be perpendicular). In short, this means:

|Angle | Position |
|:-:|-|
| 0   | New moon | 
| 90  | 1st quarter moon | 
| 180 | Full moon |
| 270 | 3rd quarter moon |

Note: the figure only plots the correct total tide for intervals of 90 degrees (0, 90, 180, or 270 degrees)

In [ ]:
# Adjust here the position of the moon (we keep the sun fixed at 0).
# Note: the figure only plots the correct total tide for intervals of 90 degrees (0, 90, 180, or 270 degrees)
angle_moon = 0

# Plotting the figure
fig, ax = plt.subplots()

dm = 2  # diameter of Earth patch in the figure
angle_sun = 0

earth = mpatches.Circle((0, 0), dm / 2, fill=True, color="C2")
lunar_tide = mpatches.Ellipse(
    (0, 0),
    width=dm + 0.75,
    height=dm + 0.15,
    angle=angle_moon,
    fill=False,
    color="k",
    linestyle="dashed",
    label="Lunar tide",
)
solar_tide = mpatches.Ellipse(
    (0, 0),
    width=dm + 0.3,
    height=dm + 0.05,
    angle=angle_sun,
    fill=False,
    color="C1",
    linestyle="dashed",
    label="Solar tide",
)
if (angle_moon - angle_sun) % 180:
    title = "Neap tide (Moon and Sun perpendicular)"
    total_tide = mpatches.Ellipse(
        (0, 0),
        width=dm + 0.8,
        height=dm + 0.45,
        angle=angle_moon,
        fill=True,
        color="C0",
        label="Total tide",
    )
else:
    title = "Spring tide (Moon and Sun in line with Earth)"
    total_tide = mpatches.Ellipse(
        (0, 0),
        width=dm + 1.05,
        height=dm + 0.2,
        angle=angle_moon,
        fill=True,
        color="C0",
        label="Total tide",
    )

ax.add_patch(total_tide)
ax.add_patch(lunar_tide)
ax.add_patch(solar_tide)
ax.add_patch(earth)
ax.text(0, 0, "Earth", ha="center", va="center", fontsize=14)
ax.set_title(title)
ax.legend()
plt.axis("equal")
plt.axis("off")
plt.tight_layout()

## 2. Tidal constituents

In the above exercise, we looked at the generation of an equilibrium tide for an Earth that is entirely covered by water, and who's gravitational field is only affected by itself, the Sun, and the Moon. In reality, our planet has continents, oceans with varying water depths, and a complex gravitational field. Hence the tidal signals we actually observe around the planet are more complex as well. Fortunately, we can break them down into multiple harmonic components, called *tidal constituents*. These can either be diurnal, semi-diurnal, short-, or long-period constituents. Each constituent is characterized by an amplitude, frequency and phase, and individually looks like a smooth sin/cosine curve. When we sum up all the individual components we get complex tidal signals, with spring-neap cycles, daily inequalities, and longer-term variations. To visualize this, we will compute the tidal signal at four different locations around the World: Scheveningen (Netherlands), Rio de Janeiro (Brazil), Galveston (Texas USA), and Jakarta (Indonesia). To do this, we will use seven harmonic components: M2, S2, K2, K1, O1, P1, and M4. Section 3.7.6. in the book gives some more information on each constituent.

First, let's have a look at the amplitudes of the constituents at each location. They are proved in the table below, in cm. Can you already determine which components will be dominant at the different locations? Also, take a look at the main lunar (M2) and solar (S2) components. In the previous exercise we determined that the lunar tide should be around 2.3 times the solar tide. Does that apply at the four locations? If not, can you think of reasons why the ratio would be different?

|Tidal amplitudes [cm]| M2 | S2 | K2 | K1 | O1 | P1 |  M4 |
|----------------|:--:|:--:|----|:--:|:--:|----|:---:|
| Scheveningen   | 75 | 18 | 6  |  8 | 12 | 3  |  21 |
| Rio de Janeiro | 30 | 17 | 5  |  6 | 10 | 2  |  3  |
| Galveston      | 13 |  4 | 1  | 16 | 16 | 5  | 0.5 |
| Jakarta        |  5 |  5 | 2  | 26 | 13 | 8  | 0.5 |

Now take a look at the function below and run the cell, we will need it in the next steps.

In [ ]:
### Functions
def harmonic(a, phi, w, t):
    """
    This function calculates the value of a harmonic function at a given time t.

    Parameters:
    a (float):      amplitude of the harmonic function.
    phi (float):    phase angle of the harmonic function.
    w (float):      angular frequency of the harmonic function.
    t (float):      time at which the function is evaluated.

    Returns:
    float:          the value of the harmonic function at the given time.
    """
    return a * np.cos(w * t + phi)

In the code below we load the data (phase and amplitude of each constituent) and plot the tidal signal at each location. For each location, the individual components are plotted (upper plot, colors) and then the combined tidal signal (lower plot, black). It is an interactive plot, you can adjust the plotted time range using the slider (from 1 day to 1 year), and you can select which tidal constituents should be plotted with the tick boxes. This way you can play around with the different constituents and see what kind of signal you get. Try to answer the following questions:

1. Set the time range to around 30 days and select only the main semi-diurnal components (M2 and S2). Which phenomenon can you detect when looking at the combined signal of these two? What is the period of this combined signal (in days)?

2. What if you take only the main diurnal components (K1 and O1). What is the main difference to the signal from before? (Hint: compute the period again, is it different from M2/S2?)

3. Use one semi-diurnal and one diurnal component (e.g. M2 and K1). Which phenomenon can we observe now? Are there differences between the four locations? Again, compute the period of the combined M2/K1 signal.

4. Let's have a look at M4, which is called a shallow-water tide. What is its period (relative to M2)? Plot only the M2 and M4 components and take a look at Scheveningen (you can see it better if you decrease the time range a bit further). What do you see there? Can you explain this? And why is Scheveningen different than the rest?

5. The combinations we looked at above have periods up to 2 weeks. But we also know that there are much longer variations. Set the time-range to 1 year and plot the combinations S2/K2 and K1/P1. What kind of combined signal can you see now? Compute the periods.

6. Finally activate all seven components and analyse the combined signals. <br>
    6.1. What are the dominant components at each location? <br>
    6.2. How would you categorize the four locations with regards to (semi)diurnality? Take a look at Figure 4.12 in the book (p. 166). Does it agree with what you can see below?

<!-- 4. Now have a look at the triangle figure below. It shows the two main diurnal and semi-diurnal components along with their periods in days. Further up the triangle, we can fill in the period of the harmonic combinations (e.g. the combined signal of the M2 and S2 has a period of 14.77 days). Can you derive the periods of the other combinations using the figure? You can also compute the periods with a formula, think back to wave groups (section 3.5.3 in the book). -->
<!-- ![periods_beats](./figures/02_periods_beats.png) -->

In [ ]:
# Tidal components and their angular frequencies (2pi divided by the period in days (= hours * 24))
comps = {
    "M2": [2 * np.pi / 12.42 * 24, "C0"],   # Principal lunar semidiurnal
    "S2": [2 * np.pi / 12 * 24, "C1"],      # Principal solar semidiurnal
    "K2": [2 * np.pi / 11.97 * 24, "C5"],   # Lunar-solar declinational semidiurnal
    "K1": [2 * np.pi / 23.93 * 24, "C2"],   # Lunar-solar declinational diurnal
    "O1": [2 * np.pi / 25.82 * 24, "C3"],   # Principal lunar diurnal
    "P1": [2 * np.pi / 24.07 * 24, "C6"],   # Principal solar diurnal
    "M4": [2 * np.pi / 6.21 * 24, "C4"],    # First shallow water lunar overtide
    # "N2": [2 * np.pi / 12.66 * 24, "C4"],   # Lunar elliptical semi-diurnal
    # "J1": [2 * np.pi / 23.1 * 24, "C4"],    # Lunar elliptical diurnal
    # "Mf": [2 * np.pi / 327.9 * 24, "C5"],   # Lunisolar fortnightly period
}

# List of the tidal components
list_comp = [comp for comp in comps]

# Time window for which we will compute the tidal components. We will use one year, 
# but in the graph we can manually adjust the plotted time window
t = np.linspace(0, 365, 20000)

# Locations that we will plot
locs = ["Scheveningen", "Rio de Janeiro", "Galveston", "Jakarta"]

# Now we load the data into a dictionary so that we have one dataframe for each component (M2, S2, ...)
tide = {}
for comp in comps.keys():
    fp = (DATA_DIR / ("02_" + comp)).with_suffix(".p")
    tide[comp] = pd.read_pickle(fp)

### Plotting widgets to make an interactive plot
# Time range slider
range_slider = widgets.IntRangeSlider(
    value=[0, int(np.max(t))],
    min=np.min(t),
    max=np.max(t),
    description="Time [d]",
    layout=widgets.Layout(width='30%')
)

# Tick boxes to select tidal components that should be plotted
M2 = widgets.Checkbox(description="M2", style={"description width": "initial"})
S2 = widgets.Checkbox(description="S2", style={"description width": "initial"})
K2 = widgets.Checkbox(description="K2", style={"description width": "initial"})
K1 = widgets.Checkbox(description="K1", style={"description width": "initial"})
O1 = widgets.Checkbox(description="O1", style={"description width": "initial"})
P1 = widgets.Checkbox(description="P1", style={"description width": "initial"})
M4 = widgets.Checkbox(description="M4", style={"description width": "initial"})
# Mf = widgets.Checkbox(description="Mf", style={"description width": "initial"})

comp_boxes = [widgets.Checkbox(description=comp, style={"description width": "initial"}) for comp in comps.keys()]

# Now we plot for each location a plot with the individual components and a plot for the total tidal water level
def plot(lims_x_axis, M2, S2, K2, K1, O1, P1, M4):
    fig, axs = plt.subplots(4, 2, sharex=True, figsize=(15, 8))
    ax = axs.flatten()

    # We first create a dictionary with the tidal components that should be plotted,
    # based on the selected tick boxes (if a box is sleected the value for that compnent is True)
    comp2plot = dict()
    if M2 == True:
        comp2plot[list(comps.keys())[0]] = list(comps.values())[0]
    if S2 == True:
        comp2plot[list(comps.keys())[1]] = list(comps.values())[1]    
    if K2 == True:
        comp2plot[list(comps.keys())[2]] = list(comps.values())[2]
    if K1 == True:
        comp2plot[list(comps.keys())[3]] = list(comps.values())[3]
    if O1 == True:
        comp2plot[list(comps.keys())[4]] = list(comps.values())[4]
    if P1 == True:
        comp2plot[list(comps.keys())[5]] = list(comps.values())[5]
    if M4 == True:
        comp2plot[list(comps.keys())[6]] = list(comps.values())[6]

    # Now we loop through the four locations (i) and the sub plots (j). Each location has two subplots, 
    # the upper showing the selected tidal components and the lower showing the combined tidal signal 
    # (sum of components).
    for i, j in zip(range(len(locs)), [0, 1, 4, 5]):
        
        # dictionary to store the harmonics
        harms = {}

        # Loop through the selected tidal components
        for comp in comp2plot.keys():

            # Store data for current component in temporary numpy array
            temp = tide[comp][tide[comp].index == locs[i].lower()].to_numpy()[0]

            # Use harmonic() to compute the harmonic funciton of the component
            harms[comp] = harmonic(temp[3], temp[2], comp2plot[comp][0], t)

            # Plot each harmonic component in the upper subplot
            ax[j].plot(t, harms[comp], "-", linewidth=1, c=comp2plot[comp][1])

        # Plot the sum of the selected constituents in the lower subplot
        if comp2plot:
            ax[j + 2].plot(t, sum([harms[comp] for comp in comp2plot.keys()]), "k", linewidth=1)

        # Axis settings
        ax[j].set_xlim(lims_x_axis)
        ax[j].set_title(locs[i])

    # Axis labels
    ax[6].set_xlabel("Time [d]")
    ax[2].set_ylabel("Tidal elevation [cm]")

    # Legend entries
    list_comp = [comp for comp in comp2plot]
    empty_str = [str("_nolegend_") for comp in comp2plot]
    list_comp += empty_str
    list_comp += ["Sum of components"]
    
    # Create legend
    fig.legend(
        list_comp,
        loc="lower center",
        ncol=len(list_comp),
        bbox_to_anchor=(0.5, -0.05),
        fontsize=14,
    )

    # Tight plot layout format
    plt.tight_layout()

# Filter potential warnings for a clean output
filterwarnings("ignore", category=UserWarning)

# Layout of the component boxes
box_layout = widgets.Layout(
    display="inline-flex",
    flex_direction="row",
    width="60%",
)

# Create the figure
figure = widgets.interactive(plot,lims_x_axis=range_slider, M2=M2, S2=S2, K2=K2, K1=K1, O1=O1, P1=P1, M4=M4)
time = figure.children[0]
controls = widgets.HBox(figure.children[1:-1], layout=box_layout)
out = figure.children[-1]
widgets.VBox([time, controls, out])

# Alternative way to create the figure (a bit quicker but looks less nice)
# widget = widgets.interact(
#     plot, lims_x_axis=range_slider, M2=M2, S2=S2, K2=K2, K1=K1, O1=O1, P1=P1, M4=M4
# )

As you can see, the different components all interact with each other to form complex time varying tidal signals, which can be very different around the globe. In the example above we included seven tidal constituents. However, the most advanced tidal models currently predict tides using up to 34 different tidal constituents. The figure below shows an example of the predicted tidal signal in Galveston, from the FES2014 tidal model, using 34 constituents. Can you see strong differences with the signal we compute for Galveston above?

![tide_galveston](./figures/02_fes_tide_prediction_sep_dec_2023.png)

### Backend

The following is not part of the student notebook but of the backend for developing the NB

In [ ]:
### DATA
# import xarray as xr
# from os.path import join

# path = '/Users/jakobchristiaa/Documents/TU Delft/PhD Turtle Beaches/git_repos/aviso-fes/data/fes2014/ocean_tide_extrapolated'
# comps = ['m2', 's2', 'k2', 'o1', 'k1', 'p1', 'm4', 'mf', 'j1', 'n2']
# locs = {
#     'scheveningen'  :   [52.125, 4.25], #lat, lon
#     'cardiff'       :   [51.5, -3.125],
#     'rio de janeiro':   [-23, -43.125],
#     'galveston'     :   [29.25, -94.6875],
#     'jakarta'       :   [-5.8125, 106.8125],
# }

# tide = {}

# # Extract constituents from FES2014 and store pickle files
# for comp in comps:
#     data = xr.open_dataset(join(path, comp + '.nc'))
#     data.coords['lon'] = (data.coords['lon'] + 180) % 360 - 180

#     # tide[comp] = pd.DataFrame(columns=)
#     temp = {}

#     for loc in locs.keys():
#         temp[loc] = (data.sel(lat=locs[loc][0],lon=locs[loc][1])
#         .to_dataframe()

#         .drop(['lat_bnds', 'lon_bnds', 'crs'], axis=1)
#         .drop(1)
#         .assign(place=loc)
#         .set_index('place', drop=True)
#     )

#     tide[comp] = pd.concat(temp.values())
#     tide[comp].to_pickle(DATA_DIR / ("02_" + comp + ".p"))

In [ ]:
### Original plotting code exercise 2 (harmonics)
# # Tidal components and their angular frequencies (numer to the right is the period in hours)
# # Comment / uncomment to select the contituents that are plotted
# comps = {
#     "M2": [2 * np.pi / 12.42, "C0"],  # Principal lunar semidiurnal
#     # 'S2'    :   [2*np.pi / 12, 'C1'],       # Principal solar semidiurnal
#     "K1": [2 * np.pi / 23.93, "C2"],  # Lunar-solar declinational diurnal
#     # 'O1'    :   [2*np.pi / 25.82, 'C3'],    # Principal lunar diurnal
#     "M4": [2 * np.pi / 6.21, "C4"],  # First shallow water lunar overtide
#     "Mf": [2 * np.pi / 327.9, "C5"],  # Lunisolar fortnightly period
# }

# # List of the tidal components
# list_comp = [comp for comp in comps]

# # Time window for which we will compute the tidal components. t is in hours so for one month we use around 300 hours (Wouldn't it be 720 hrs?).
# t = np.linspace(0, 338, 1000)

# # First we load the data (tidal constituents) into a dictionary so that we have one dataframe for each constituent (M2, S2, ...)
# tide = {}
# for comp in comps.keys():
#     fp = (DATA_DIR / ("02_" + comp)).with_suffix(".p")
#     tide[comp] = pd.read_pickle(fp)

# # Locations that we will plot
# locs = ["Scheveningen", "Rio de Janeiro", "Galveston", "Jakarta"]


# range_slider = widgets.FloatRangeSlider(
#     value=[0, np.max(t)],
#     min=np.min(t),
#     max=np.max(t),
#     step=0.1,
#     description="x-axis:",
#     readout_format=".1f",
# )


# def plot(lims_x_axis):
#     # Now we plot for each location a plot with the individual components and a plot for the total tidal water level
#     fig, axs = plt.subplots(4, 2, sharex=True, figsize=(15, 8))
#     ax = axs.flatten()

#     for i, j in zip(range(len(locs)), [0, 1, 4, 5]):
#         harms = {}
#         for comp in comps.keys():
#             # Store data for current component in temporary numpy array
#             temp = tide[comp][tide[comp].index == locs[i].lower()].to_numpy()[0]

#             # Use harmonic() to compute each harmonic constituent
#             harms[comp] = harmonic(temp[3], temp[2], comps[comp][0], t)

#             # Plot each individual constituent in the upper subplot
#             ax[j].plot(t, harms[comp], "-", linewidth=1, c=comps[comp][1])

#         # Plot the sum of the selected constituents in the lower subplot
#         ax[j + 2].plot(t, sum([harms[comp] for comp in comps.keys()]), "k")

#         # Axis settings
#         ax[j].set_xlim(lims_x_axis)
#         ax[j].set_title(locs[i])

#     ax[6].set_xlabel("Time [h]")
#     ax[2].set_ylabel("Tidal elevation [cm]")

#     list_comp = [comp for comp in comps]
#     empty_str = [str("_nolegend_") for comp in comps]
#     list_comp += empty_str
#     list_comp += ["Sum of components"]

#     fig.legend(
#         list_comp,
#         loc="lower center",
#         ncol=len(list_comp),
#         bbox_to_anchor=(0.5, -0.05),
#         fontsize=14,
#     )

#     plt.tight_layout()


# filterwarnings("ignore", category=UserWarning)

# widgets.interact(plot, lims_x_axis=range_slider)

In [ ]:
### Edited code for exercise 2 figure (not yet working)

# # Tidal components and their angular frequencies (2pi divided by the period in days (= hours * 24))
# comps = {
#     "M2": [2 * np.pi / 12.42 * 24, "C0"],   # Principal lunar semidiurnal
#     "S2": [2 * np.pi / 12 * 24, "C1"],      # Principal solar semidiurnal
#     "K2": [2 * np.pi / 11.97 * 24, "C5"],   # Lunar-solar declinational semidiurnal
#     "K1": [2 * np.pi / 23.93 * 24, "C2"],   # Lunar-solar declinational diurnal
#     "O1": [2 * np.pi / 25.82 * 24, "C3"],   # Principal lunar diurnal
#     "P1": [2 * np.pi / 24.07 * 24, "C6"],   # Principal solar diurnal
#     "M4": [2 * np.pi / 6.21 * 24, "C4"],    # First shallow water lunar overtide
#     # "Mf": [2 * np.pi / 327.9 * 24, "C5"],   # Lunisolar fortnightly period
# }

# # Time window for which we will compute the tidal components. We will use one year, 
# # but in the graph we can manually adjust the plotted time window
# t = np.linspace(0, 365, 20000)

# # Locations that we will plot
# locs = ["Scheveningen", "Rio de Janeiro", "Galveston", "Jakarta"]

# # Now we load the data into a dictionary so that we have one dataframe for each component (M2, S2, ...)
# tide = {}
# for comp in comps.keys():
#     fp = (DATA_DIR / ("02_" + comp)).with_suffix(".p")
#     tide[comp] = pd.read_pickle(fp)

# ### Plotting widgets to make an interactive plot
# # Time range slider
# range_slider = widgets.IntRangeSlider(
#     value=[0, int(np.max(t))],
#     min=np.min(t),
#     max=np.max(t),
#     description="Time [d]",
# )

# # Tick boxes to select tidal components that should be plotted
# comp_boxes = [widgets.Checkbox(description=comp, style={"description width": "initial"}) for comp in comps.keys()]

# # Now we plot for each location a plot with the individual components and a plot for the total tidal water level
# def plot(lims_x_axis, comp_vals):
#     fig, axs = plt.subplots(4, 2, sharex=True, figsize=(15, 8))
#     ax = axs.flatten()

#     # We first create a dictionary with the tidal components that should be plotted,
#     # based on the selected tick boxes (if a box is sleected the value for that compnent is True)
#     comp2plot = dict()
#     for i, comp in zip(range(len(comps)), comp_vals):
#         if comp == True:
#             comp2plot[list(comps.keys())[i]] = list(comps.values())[i]

#     # Now we loop through the four locations (i) and the sub plots (j). Each location has two subplots, 
#     # the upper showing the selected tidal components and the lower showing the combined tidal signal 
#     # (sum of components).
#     for i, j in zip(range(len(locs)), [0, 1, 4, 5]):
        
#         # dictionary to store the harmonics
#         harms = {}

#         # Loop through the selected tidal components
#         for comp in comp2plot.keys():

#             # Store data for current component in temporary numpy array
#             temp = tide[comp][tide[comp].index == locs[i].lower()].to_numpy()[0]

#             # Use harmonic() to compute the harmonic funciton of the component
#             harms[comp] = harmonic(temp[3], temp[2], comp2plot[comp][0], t)

#             # Plot each harmonic component in the upper subplot
#             ax[j].plot(t, harms[comp], "-", linewidth=0.8, c=comp2plot[comp][1])

#         # Plot the sum of the selected constituents in the lower subplot
#         if comp2plot:
#             ax[j + 2].plot(t, sum([harms[comp] for comp in comp2plot.keys()]), "k", linewidth=1)

#         # Axis settings
#         ax[j].set_xlim(lims_x_axis)
#         ax[j].set_title(locs[i])

#     # Axis labels
#     ax[6].set_xlabel("Time [d]")
#     ax[2].set_ylabel("Tidal elevation [cm]")

#     # Legend entries
#     list_comp = [comp for comp in comp2plot]
#     empty_str = [str("_nolegend_") for comp in comp2plot]
#     list_comp += empty_str
#     list_comp += ["Sum of components"]
    
#     # Create legend
#     fig.legend(
#         list_comp,
#         loc="lower center",
#         ncol=len(list_comp),
#         bbox_to_anchor=(0.5, -0.05),
#         fontsize=14,
#     )

#     # Tight plot layout format
#     plt.tight_layout()

# # Filter potential warnings for a clean output
# filterwarnings("ignore", category=UserWarning)

# # Layout of the component boxes
# box_layout = widgets.Layout(
#     display="inline-flex",
#     flex_direction="row",
#     width="60%",
# )

# # Create the figure
# figure = widgets.interactive(plot,lims_x_axis=range_slider, comp_vals=comp_boxes)
# time = figure.children[0]
# controls = widgets.HBox(figure.children[1:-1], layout=box_layout)
# out = figure.children[-1]
# widgets.VBox([time, controls, out])

# # Alternative way to create the figure (a bit quicker but looks less nice)
# # widget = widgets.interact(
# #     plot, lims_x_axis=range_slider, M2=M2, S2=S2, K2=K2, K1=K1, O1=O1, P1=P1, M4=M4
# # )